In [2]:
#utf-8
# coding: utf-8
import sys
import pygame
import math
import vtk
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
from stl import mesh
import serial
import time
import threading

pygame 2.5.0 (SDL 2.28.0, Python 3.10.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'OpenGL'

In [10]:
# Global variables for angle values
pitch_angle = 0
roll_angle = 0
yaw_angle = 0

# Global variables for initial values
initialPitchAngle = 0
initialRollAngle = 0 
initialYawAngle = 0

# Serial port configuration
SERIAL_PORT = 'COM10'
BAUD_RATE = 115200

In [11]:
# Function to receive gyro values from serial port
def receive_angle_values():
    global pitch_angle, roll_angle, yaw_angle
    global initialPitchAngle, initialRollAngle, initialYawAngle
    # Open serial port
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE)

    while True:
        try:
        # Read accelerometer values from serial port
            line = ser.readline().decode().strip()
            if line.startswith("X:"):
                #print("Accelerometer: ", line)
                values = line.split(' ')
                accel_x = float(''.join(filter(lambda x: x.isdigit() or x == '-', values[0])))+(0.110)
                accel_y = float(''.join(filter(lambda x: x.isdigit() or x == '-', values[1])))+(-0.008)
                accel_z = float(''.join(filter(lambda x: x.isdigit() or x == '-', values[2])))+(3.050)

                # Calculate pitch angle (rotation around x-axis)
                pitch_angle = math.degrees(math.atan2(accel_y, accel_z))
                # Calculate roll angle (rotation around y-axis)
                roll_angle = math.degrees(math.atan2(accel_x, accel_z))
                # Calculate yaw angle (rotation aroun z-axis)
                yaw_angle = math.degrees(math.atan2(accel_x, accel_y))

                #Calibration - set initial values as zero point
                if initialPitchAngle == 0 and initialRollAngle == 0 and initialYawAngle == 0:
                    initialPitchAngle = pitch_angle
                    initialRollAngle = roll_angle
                    initialYawAngle = yaw_angle
                    print("Calibrado")

                pitch_angle = pitch_angle - initialPitchAngle
                roll_angle = roll_angle - initialRollAngle
                yaw_angle = yaw_angle - initialYawAngle
                #print("Angulosserial: ", pitch_angle, roll_angle, yaw_angle)

        except:
            pass
        time.sleep(0.5)

    ser.close()

#receive_angle_values()

In [12]:
# Function to load and initialize the STL mesh
def load_stl_mesh():
    return mesh.Mesh.from_file('xyzCube.stl')

# Function to initialize Pygame and OpenGL
def init():
    pygame.init()
    width, height = 1000, 800
    pygame.display.set_mode((width, height), OPENGL | DOUBLEBUF)
    gluPerspective(45, (width / height), 0.1, 50.0)
    glTranslatef(0.0, 0.0, -10)  # Adjust the object's position
    glScalef(0.01, 0.01, 0.01)  # Adjust the object's scale
    glRotatef(-90, 1, 0, 0)     # Adjust the object's initial rotation

# Function to handle key events
def handle_events():
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

# Function to update the rotation of the STL object based on gyro values
def update_rotation():
    global pitch_angle, roll_angle, yaw_angle

    glLoadIdentity()
    gluLookAt(0, 0, 10, 0, 0, 0, 0, 1, 0) # Set the camera position

    glRotatef(pitch_angle, 1, 0, 0) # Rotate the object around x-axis
    glRotatef(roll_angle, 0, 1, 0)  # Rotate the object around y-axis
    glRotatef(yaw_angle, 0, 0, 1)   # Rotate the object around z-axis

    
# Function to render the STL object
def render_stl(stl_mesh):
    glEnable(GL_DEPTH_TEST)
    glDepthFunc(GL_LEQUAL)
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glEnable(GL_COLOR_MATERIAL)
    glClearColor(0.2, 0.2, 0.2, 1) # Set background color
    glColor3f(1, 1, 1) # Set object color
    
    glMaterialfv(GL_FRONT, GL_AMBIENT, (1.0, 1.0, 1.0, 1.0))
    glColor(1.0, 0.0, 0.0, 1.0)

    # glMaterialfv(GL_FRONT, GL_DIFFUSE, (0.8, 0.8, 0.8, 1))
    # glMaterialfv(GL_FRONT, GL_SPECULAR, (1, 1, 1, 1))
    # glMaterialfv(GL_FRONT, GL_SHININESS, 50)

    glBegin(GL_TRIANGLES)
    for triangle in stl_mesh.vectors:
        for vertex in triangle:
            glVertex3fv(vertex)
    glEnd()
    
    pygame.display.flip()
    #pygame.time.wait(10)

# Main function
def main():
    init()
    stl_mesh = load_stl_mesh()

    # Start receiving angle values in a separate thread
    angle_values = threading.Thread(target=receive_angle_values)
    angle_values.start()

    while True:
        handle_events()
        update_rotation()
        render_stl(stl_mesh)
        pygame.time.wait(10)

if __name__ == "__main__":
    main()

GLError: GLError(
	err = 1282,
	description = b'opera\xe7\xe3o inv\xe1lida',
	baseOperation = glEnd,
	cArguments = ()
)

Calibrado


In [ ]:
# Function to load and initialize the STL mesh
def load_stl_mesh(filename):
    stl_mesh = mesh.Mesh.from_file(filename)
    return stl_mesh

# Function to initialize Pygame and OpenGL
def init(width, height):
    pygame.init()
    pygame.display.set_mode((width, height), DOUBLEBUF | OPENGL)
    gluPerspective(45, (width / height), 0.1, 50.0)
    glTranslatef(0.0, 0.0, -5.0)
    glRotatef(-90, 1, 0, 0)  # Adjust the initial rotation if necessary

# Function to handle key events
def handle_events():
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            quit()

# Function to update the rotation based on gyro values
def update_rotation():
    global pitch_angle, roll_angle, yaw_angle
    
    glRotatef(pitch_angle, 1, 0, 0)
    glRotatef(yaw_angle, 0, 1, 0)
    glRotatef(roll_angle, 0, 0, 1)

# Function to render the STL object
def render_stl(stl_mesh):
    glBegin(GL_TRIANGLES)
    for triangle in stl_mesh.vectors:
        for vertex in triangle:
            glVertex3fv(vertex)
    glEnd()

# Main function
def main():
    # Load the STL mesh
    stl_mesh = load_stl_mesh('xyzCube.stl')

    # Start receiving gyro values in a separate thread
    gyro_thread = threading.Thread(target=receive_angle_values)
    gyro_thread.start()

    # Set up the Pygame and OpenGL environment
    width, height = 800, 600
    init(width, height)

    while True:
        handle_events()

        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glEnable(GL_DEPTH_TEST)
        glEnable(GL_LIGHTING)
        glEnable(GL_LIGHT0)
        glEnable(GL_COLOR_MATERIAL)
        glClearColor(0.2, 0.2, 0.2, 1.0)

        update_rotation()
        render_stl(stl_mesh)

        pygame.display.flip()
        pygame.time.wait(10)

if __name__ == "__main__":
    main()